## Wrappers for the Scikit-Learn API

You can use Sequential Keras models (single-input only) as part of your Scikit-Learn workflow via the wrappers found at keras.wrappers.scikit_learn.py.

There are two wrappers available:

keras.wrappers.scikit_learn.KerasClassifier(build_fn=None, **sk_params), which implements the Scikit-Learn classifier interface,

keras.wrappers.scikit_learn.KerasRegressor(build_fn=None, **sk_params), which implements the Scikit-Learn regressor interface.

#### Arguments

* build_fn: callable function or class instance
* sk_params: model parameters & fitting parameters

build_fn should construct, compile and return a Keras model, which will then be used to fit/predict. One of the following three values could be passed to build_fn:

1. A function
2. An instance of a class that implements the call method
3. None. This means you implement a class that inherits from either KerasClassifier or KerasRegressor. The call method of the present class will then be treated as the default build_fn.

sk_params takes both model parameters and fitting parameters. Legal model parameters are the arguments of build_fn. Note that like all other estimators in scikit-learn, 'build_fn' should provide default values for its arguments, so that you could create the estimator without passing any values to sk_params.

sk_params could also accept parameters for calling fit, predict, predict_proba, and score methods (e.g., epochs, batch_size). fitting (predicting) parameters are selected in the following order:

1. Values passed to the dictionary arguments of fit, predict, predict_proba, and score methods
2. Values passed to sk_params
3. The default values of the keras.models.Sequential fit, predict, predict_proba and score methods

When using scikit-learn's grid_search API, legal tunable parameters are those you could pass to sk_params, including fitting parameters. In other words, you could use grid_search to search for the best batch_size or epochs as well as the model parameters.

In [1]:
from keras.models import Sequential
from keras.layers import Dense, Activation

def build_model(optimizer='rmsprop', dense_dims=32):
    model = Sequential()
    model.add(Dense(dense_dims, activation='relu', input_dim=100))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer=optimizer,
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    
    return model
    

Using TensorFlow backend.


In [2]:
from keras.wrappers.scikit_learn import KerasClassifier

keras_classifier = KerasClassifier(build_model, epochs=2)

In [3]:
import numpy as np
data = np.random.random((1000, 100))
labels = np.random.randint(2, size=(1000, 1))

keras_classifier.fit(data, labels)

Epoch 1/2
1000/1000 [==============================] - 0s - loss: 0.7050 - acc: 0.5130     
Epoch 2/2
1000/1000 [==============================] - 0s - loss: 0.6943 - acc: 0.5230     


In [4]:
keras_classifier.predict_proba(data[:2])

2/2 [==============================] - 0s


array([[ 0.47725362,  0.52274638],
       [ 0.50770903,  0.49229094]], dtype=float32)

In [5]:
from sklearn.model_selection import GridSearchCV

gs = GridSearchCV(keras_classifier, {'epochs': [2, 3], 'dense_dims':[16, 32]})

In [6]:
gs.fit(data, labels)

Epoch 1/2
666/666 [==============================] - 0s - loss: 0.7206 - acc: 0.4835     
Epoch 2/2
667/667 [==============================] - 0s - loss: 0.7147 - acc: 0.4828     
Epoch 2/2
667/667 [==============================] - 0s - loss: 0.7088 - acc: 0.5097     
Epoch 2/2
666/666 [==============================] - 0s - loss: 0.7032 - acc: 0.4925     
Epoch 2/3
666/666 [==============================] - 0s - loss: 0.6962 - acc: 0.5180     
Epoch 3/3
667/667 [==============================] - 0s - loss: 0.7193 - acc: 0.4873     
Epoch 2/3
667/667 [==============================] - 0s - loss: 0.7066 - acc: 0.4933     
Epoch 3/3
667/667 [==============================] - 0s - loss: 0.7499 - acc: 0.4783     
Epoch 2/3
667/667 [==============================] - 0s - loss: 0.7152 - acc: 0.4858     
Epoch 3/3
666/666 [==============================] - 0s - loss: 0.7093 - acc: 0.4940     
Epoch 2/2
667/667 [==============================] - 0s - loss: 0.7112 - acc: 0.4948     
Epoch 2/2


GridSearchCV(cv=None, error_score='raise',
       estimator=<keras.wrappers.scikit_learn.KerasClassifier object at 0x106972710>,
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'epochs': [2, 3], 'dense_dims': [16, 32]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=None, verbose=0)

In [7]:
gs.best_params_

{'dense_dims': 16, 'epochs': 3}